# Fun with Python
## or: Python Tricks, Features and Quirks

Robert Speck, July 5, 2019, Jülich Supercomputing Centre


### Before we start.. a disclaimer

* You may already know some/many of the things we discuss here
* Some will be boring, others will be cool. Some will be quick, others will take time. Some wil be easy, others will leave you confused.
* Since this is the first iteration:
 - This could take 60 or 100 minutes
 - Not all of this may work/will work in the future
* This is a personal and biased list
* Sorting is close to random, but only close
* Python 3.6+ only, see https://python3statement.org/ 


### References

Most of the things here are based on three resources:

* the book "[Python Tricks: A Buffet of Awesome Python Features](https://www.amazon.de/Python-Tricks-Buffet-Awesome-Features/dp/1775093301)" 
by Dan Bader, October 2017
* The internet, in particular:
 - https://realpython.com/ and its "Python Tricks", Tutorials and the awesome newsletter
 - https://stackoverflow.com and the Python tag
 - https://www.google.de/
* my own, sometimes painful experience of 10 years of Python (but no programming course) 


### Setting up an environment

We are going to use [conda](https://conda.io) for managing our Python installation.

<div class="alert alert-success">
<strong>Lesson learned</strong>: Never use the system python. Never add stuff to the system python. Always use a virtual environment.
</div>



After installing e.g. miniconda, run

```
conda create --name fun_with_python python=3.7
conda activate fun_with_python
conda install pip jupyter numpy Cython numba
```
Did it work for this notebook?


In [1]:
import sys
print(sys.executable)


/miniconda3/envs/fun_with_python/bin/python


All set, let's get started









## 1. Assertions, but do it right

Let's start simple:

In [2]:
def apply_discount(product, discount):
    price = int(product['price']) * (1.0 - discount)
    assert (0 <= price <= product['price'], 'Go back to school, buddy!')
    return price

<>:3: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:3: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:3: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-2-0863961bedfa>:3: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (0 <= price <= product['price'], 'Go back to school, buddy!')


In [3]:
shoes = {'name': 'Fancy Shoes', 'price': 100}
apply_discount(shoes, 0.25)

75.0

In [4]:
apply_discount(shoes, 2.0)

-100.0

Wait, what? Didn't we just add an assert statement to catch these thing?




In [ ]:
assert(False, 'This should be always there!')

<div class="alert alert-success">
<strong>Lesson learned</strong>: Use dedicated IDEs and linters to catch really nasty bugs before they make it into you code. Assertions can be disabled in multiple ways (PYTHONOPTIMIZE, -O, PYTHONWARNINGS)! 
</div>


Funny, isn't it? How could you know? But imagine, you had something like this:


In [ ]:
def delete(product, user):
    assert user['can_delete']
    product['out_of_stock'] = True

In [ ]:
admin = {'name': 'Alice', 'can_delete': True}
delete(shoes, admin)
print(shoes)

Disabling assertions (accidentally or on purpose) would remove any "security measures" from this code. Thus:

<div class="alert alert-success">
<strong>Lesson learned</strong>: Use assertions to help debugging and to catch rare/weird bugs (Heisenbugs, Schrödinbugs etc.), but not for catching run-time errors or for protecting your code. Use customized exceptions there instead.
</div>

## 2. Dictionaries and what they tell us about Python

Let's get some dictionary for data:

In [1]:
data = dict()

In [2]:
import sys
sys.getsizeof(data)

240

In [3]:
data = {'a': 1}

In [4]:
sys.getsizeof(data)

240

Interesting, right? Why doesn't the mamory consumption grow when data is added to a dictionary? 

Let's see:

In [6]:
data = {}
for one_letter in 'abcdefghijklmnopqrstuvwxyz':
        data[one_letter] = one_letter
        print(f'{len(data)}, sys.getsizeof(data) = {sys.getsizeof(data)}')

1, sys.getsizeof(data) = 240
2, sys.getsizeof(data) = 240
3, sys.getsizeof(data) = 240
4, sys.getsizeof(data) = 240
5, sys.getsizeof(data) = 240
6, sys.getsizeof(data) = 368
7, sys.getsizeof(data) = 368
8, sys.getsizeof(data) = 368
9, sys.getsizeof(data) = 368
10, sys.getsizeof(data) = 368
11, sys.getsizeof(data) = 648
12, sys.getsizeof(data) = 648
13, sys.getsizeof(data) = 648
14, sys.getsizeof(data) = 648
15, sys.getsizeof(data) = 648
16, sys.getsizeof(data) = 648
17, sys.getsizeof(data) = 648
18, sys.getsizeof(data) = 648
19, sys.getsizeof(data) = 648
20, sys.getsizeof(data) = 648
21, sys.getsizeof(data) = 648
22, sys.getsizeof(data) = 1184
23, sys.getsizeof(data) = 1184
24, sys.getsizeof(data) = 1184
25, sys.getsizeof(data) = 1184
26, sys.getsizeof(data) = 1184


Aha! Python allocates dictionary memory in chunks. This makes costly memory allocations rare!

So, let's clear this dictionary and see where we end up:

In [8]:
data.clear()
sys.getsizeof(data)

72

Ha! So making a new dictionary already allocattes some memory for its content, while clearing removes also the allocated memory. Who would have guessed that?

Anyway, let's get started with dictionaries!

In [9]:
data = {1: 'Alice', 2: 'Bob'}

In [10]:
def get_name(id):
    return data[id]

In [11]:
get_name(1)


'Alice'

In [12]:
get_name(4)

KeyError: 4

Hm.. that should never happen, right? So, we got to prevent this:

In [ ]:
def get_name(id):
    if id in data:
        return data[id]
    else:
        return 'Not found'

In [ ]:
get_name(1), get_name(4)

OK, works, but we have to query the dictionary twice. Very inefficient and not very Pythonic, let's do better:

In [ ]:
def get_name(id):
    try:
        return data[id]
    except KeyError:
        return 'Not found'

In [ ]:
get_name(2), get_name(4)

<div class="alert alert-success">
    <strong>Lesson learned</strong>: This is called EAFP coding style: "easier to ask for forgiveness than permission".
</div>

We can even do better, more Pythonic:

In [ ]:
def get_name(id):
    return data.get(id, 'Not found')

In [ ]:
get_name(2), get_name(4)

<div class="alert alert-success">
<strong>Lesson learned</strong>: Use exception handling or built-in functions wherever possible.
</div>

Btw. what will happen if I do this:

In [ ]:
get_name(True)

Weird, isn't it?

In [ ]:
print(True == 1, True == 2, True == 1.0, True == 1.0+0.0j, False == 0, False == 0.0)

<div class="alert alert-success">
    <strong>Lesson learned</strong>: <i>bool</i> is a subclass of <i>int</i> and can be used like this.
</div>

OK, good to know, what how about this thing?

In [ ]:
data = {True: 'yes', 1: 'no'}

In [ ]:
data

Really weird! Why is that? Let's break it down:

In [ ]:
data = dict()
print(data)

In [ ]:
data[True] = 'yes'
print(data)

In [ ]:
data[1] = 'no'
print(data)

In [ ]:
data[1] = 'maybe'
print(data)

Aha! It looks like since True == 1, the key does not change. Python (or, at least CPython) only updates the value, but keeps the first key for performance. But is equality the sole criterion? Let's find out:

In [ ]:
import numpy as np

class AlwaysEquals():
    # overload equality check to return always True
    def __eq__(self, other):
        return True
    # overload hash operator to return always a different hash
    def __hash__(self):
        return 42 + np.random.randint(1, 100)
    
a = AlwaysEquals()
b = AlwaysEquals()

In [ ]:
print(f'a and b are equal? {a == b}')
print(f'hashes of a and b are equal?: {hash(a) == hash(b)}\n')

In [ ]:
data = {a: 'yes', b: 'no'}
print(f'resulting dictionary:\n {data}')

<div class="alert alert-success">
    <strong>Lesson learned</strong>: Testing whether a key is in a dictionary relies on equality of the objects <strong>and</strong> their hashes. 
</div>

OK, back to normal Python stuff. Let's merge dictionaries!

In [ ]:
d1 = {'a': 1, 'c': 3}
d2 = {'b': 2, 'c': 4}

In [ ]:
result = {}
result.update(d1)
print(result)
result.update(d2)
print(result)

OK, simple. Note that the order of updating is (not surpringly) important for dealing with conflicts. Let's make this more Pythonic:

In [ ]:
result = dict(d1, **d2)
print(result)
result = dict(d2, **d1)
print(result)

Great! The **\**-operator** for 'unpacking' makes life easier here! How about merging more than 2?

In [ ]:
d3 = {'d': 4}
result = dict(d1, **d2, **d3)
print(result)

So, what is this unpacking operator, exactly?

## 3. \*args and \*\*kwargs


You will find these things in many public Python codes, in particular in libraries and codes which others should use. So, let's see what they do.


In [5]:
def foo(required, *args, **kwargs):
    print(required)
    if args:
        print(args)
    if kwargs:
        print(kwargs)

In [7]:
foo(1)

1


In [9]:
foo(1, 2)

1
(2,)


In [10]:
foo(1, 2, 3)

1
(2, 3)


In [11]:
foo(1, 2, 3, key='hello')

1
(2, 3)
{'key': 'hello'}


<div class="alert alert-success">
    <strong>Lesson learned</strong>: *args gathers positional arguments as a tuple, **kwargs keyword arguments as a dictionary. The asterix are the relevant symbols here, calling them args and kwargs is just a convention. 
</div>

So why whould I use that? Imagine you are writing a library other should use and extend:

In [ ]:
class Car():
    def __init__(self, color, mileage, *args, **kwargs):
        self.color = color
        self.mileage = mileage
        self.features = args
        self.__dict__.update(kwargs)
    
class BlueFancyCar(Car):
    def __init__(self, mileage, *args, cost):
        super().__init__('blue', mileage, *args, cost=cost)

In [ ]:
c = Car('red', 0, status='broken')
print(f'This car is {c.status}')
b = BlueFancyCar(0, 'a', 'b', cost=1E06)
print(f'This car has {b.features} and costs {b.cost} euros')

<div class="alert alert-success">
    <strong>Lesson learned</strong>: *args and **kwargs are really helpful, but can obfuscate signature. Use with care and documentation!
</div>

Other nice things to do with unpacking arguments:

In [12]:
def print_vector(x, y, z):
    print(f'<{x},{y},{z}>')

In [13]:
print_vector(0, 1, 0)

<0,1,0>


In [14]:
list_vector = [0, 1, 0]
print_vector(list_vector)

TypeError: print_vector() missing 2 required positional arguments: 'y' and 'z'

In [15]:
print_vector(*list_vector)

<0,1,0>


In [16]:
tuple_vector = (0, 1, 0)
print_vector(*tuple_vector)

<0,1,0>


In [17]:
gen_vector = (x % 2 for x in range(3))
print_vector(*gen_vector)

<0,1,0>


In [ ]:
dict_vector = {'x': 0, 'y': 1, 'z': 0}
print_vector(**dict_vector)

Neat, huh?

## 4. (Not much) fun with cloning objects

A variable and an object are different things in Python. This is obvious in cases like:

In [ ]:
listA = [0]
listB = listA
listB.append(1)
print(listA)

How about this case?

In [ ]:
class Foo():
    def __init__(self, data):
        self.data = data

data = 'blue'
c = Foo(data)
print(c.data, data)

In [ ]:
c.data = 'red'
print(c.data, data)

In [ ]:
print(c.data is data)
print(id(c.data), id(data))

Not suprising, right? The objects are **immutable**, all is well. But what about this one?

In [ ]:
data = [0, 1, 2]
c = Foo(data)
print(c.data, data)
c.data[0] = 'XXX'
print(c.data, data)

Uh, oh! What the..?

In [ ]:
print(c.data is data)
print(id(c.data), id(data))

<div class="alert alert-success">
    <strong>Lesson learned</strong>: Python is "pass-by-object-reference", i.e. a function receives a reference to (and will access) the same object in memory as used by the caller. However, it does not receive the box that the caller is storing this object in, it will create a new, local variable for it.
</div>

So, what can we do about it? We need some form of cloning!

In [ ]:
import copy as cp
class Foo_Clone():
    def __init__(self, data):
        self.data = cp.copy(data)

In [ ]:
data = [0, 1, 2]
c = Foo_Clone(data)
print(c.data, data)
c.data[0] = 'XXX'
print(c.data, data)

Hey, cool, that's it! Next topic! ... wait, no?

In [ ]:
data = [[0, 0, 0], [1, 1, 1], [2, 2, 2]]
c = Foo_Clone(data)
print(c.data, data)
c.data[0][0] = 'XXX'
print(c.data, data)

In [ ]:
print(c.data is data)
print(c.data[0] is data[0])

<div class="alert alert-success">
    <strong>Lesson learned</strong>: The standard copy operator (like, instaniating new lists or using .copy etc.) only makes <strong>shallow copies</strong> of objects, i.e. it creates a copy of an object and populates it with references to the child objects. If the object is immutable, this is not an issues.
</div>

So, what can we do?

In [ ]:
class Foo_DeepClone():
    def __init__(self, data):
        self.data = cp.deepcopy(data)

In [ ]:
data = [[0, 0, 0], [1, 1, 1], [2, 2, 2]]
c = Foo_DeepClone(data)
print(c.data, data)
c.data[0][0] = 'XXX'
print(c.data, data)

In [ ]:
print(c.data is data)
print(c.data[0] is data[0])

<div class="alert alert-success">
    <strong>Lesson learned</strong>: The deepcopy operator copies objects recursively!
</div>

But: there are some objects, which cannot be copied via deepcopy:

In [ ]:
f = open('test.txt', 'w')
c = Foo_DeepClone(f)
f.close()

In [ ]:
f.close()

Seems obscure, but things like MPI communicators can also not be serialized! You encounter this for many, more complex libraries, too. You could try the dill package to circumvent this or write your own \_\_copy\_\_ function. 

## 5. One aspect of \_\_init\_\_ vs. \_\_new\_\_

Say you want to write a code which adds "stuff". You don't know what kind of stuff users will want to add, so you ask them to wrap their data in a data type. This ensures that your code works with a defined data type and operations on it.

This is the awesome code you wrote:

In [18]:
class wrapper():
    def __init__(self, data):
        self.values = data
        self.status = 'AWESOME'

def my_cool_adder(data1, data2):
    return data1 + data2

This is what the user writes:

In [19]:
class mylist(wrapper):
    def __add__(self, other):        
        return mylist([sum(x) for x in zip(self.values, other.values)])

Let's try this:

In [20]:
d1 = mylist([0, 1, 2])
d2 = mylist([3, 4, 5])
d3 = my_cool_adder(d1, d2)
print(d3)
print(d3.values)
print(d3.status)

[3, 5, 7]
AWESOME


Works, so how about dictionaries?

In [21]:
class mydict(wrapper):
    def __add__(self, other):
        return mydict({k: self.values[k] + other.values[k] for k in self.values.keys() & other.values})

In [22]:
d1 = mydict({'a': 0, 'b': 1, 'c': 2})
d2 = mydict({'a': 3, 'b': 4, 'c': 5})
d3 = my_cool_adder(d1, d2)
print(d3)
print(d3.values)
print(d3.status)

{'b': 5, 'c': 7, 'a': 3}
AWESOME


So, what if we want to use things like a Numpy array? Couldn't we use it directly, like:

In [23]:
import numpy as np
d1 = np.array([0, 1, 2])
d2 = np.array([3, 4, 5])
d3 = my_cool_adder(d1, d2)
print(d3)
print(d3.values)
print(d3.status)

[3 5 7]


AttributeError: 'numpy.ndarray' object has no attribute 'values'

Ah, uh, this does not return anything I could use in my code, I'm leaving the data type here. Do I have to do it like this?

In [24]:
class myarray(wrapper):
    def __add__(self, other):
        return myarray(self.values + other.values)

In [25]:
d1 = myarray(np.array([0, 1, 2]))
d2 = myarray(np.array([3, 4, 5]))
d3 = my_cool_adder(d1, d2)
print(d3)
print(d3.values)
print(d3.status)

[3 5 7]
AWESOME


Rather inconvenient, isn't it? Just overriding the addition to cast it as `myarray`.. there should be a better way to do that! There is:

In [26]:
class mynewarray(np.ndarray):
    def __new__(cls, data):
        obj = super().__new__(cls, data.shape)
        obj[:] = data[:]
        obj.status = 'AWESOME'
        return obj
    # Need to add this to make new objects coming from +, -, ... to have the status attribute
    def __array_finalize__(self, obj):
        self.status = getattr(obj, 'status', None)    

Note that we do not need to specify how to add here, because the parent class already knows how to do that! We also do not need the values attribute anymore, we can work directly on top of the Numpy array.

In [28]:
d1 = mynewarray(np.array([0, 1, 2]))
d2 = mynewarray(np.array([3, 4, 5]))
d3 = my_cool_adder(d1, d2)
print(d3)
print(d3.status)

[3. 5. 7.]
AWESOME


<div class="alert alert-success">
    <strong>Lesson learned</strong>: Use __new__ when you need to control the creation of a new instance. Use __init__ when you need to control initialization of a new instance. __new__ is the first step of instance creation. It's called first, and is responsible for returning a new instance of your class. In contrast, __init__ doesn't return anything; it's only responsible for initializing the instance after it's been created.
</div>

## 6. Abstract classes, freezing..

OK, so you can add stuff. But how to enforce people to actually implement or provide the \_\_add\_\_ functionality? Or, even, to inform them that you need this? Like this?

In [29]:
class wrapper():
    def required_function(self):
        raise NotImplementedError()

In [30]:
class mytype(wrapper):
    def other_function(self):
        print('not me')
    pass

In [31]:
m = mytype()

In [32]:
m.other_function()

not me


In [33]:
m.required_function()

NotImplementedError: 

Well, that works, but only when actually calling required\_function we see that there is something wrong. Inheriting from the wrapper class is fine. Can we do better?

In [34]:
from abc import ABCMeta, abstractmethod

class abstract_wrapper(metaclass=ABCMeta):
    @abstractmethod
    def required_function():
        pass

In [35]:
class mytype(abstract_wrapper):
    pass

In [36]:
m = mytype()

TypeError: Can't instantiate abstract class mytype with abstract methods required_function

<div class="alert alert-success">
    <strong>Lesson learned</strong>: To catch nasty bugs in your class hierarchy as early as possible, use Abstract Base Classes (ABCs) with abstract methods. 
</div>

Since we are alrady talking about nasty bugs in classes, let me show you this one, straight from my own experience:

In [37]:
class Foo():
    def __init__(self):
        self.fluorescence = None
        
    def get_surprise(self):
        if self.fluorescence is None:
            number = np.random.rand(1)
            self.flourescence = number
        else:
            pass

In [38]:
c = Foo()
print(c.fluorescence)

None


In [39]:
c.get_surprise()
print(c.fluorescence)

None


Huh? Didn't we just set the attibute to some random number? Is was None, but still is after the function call.. what's going on?

To avoid these sometimes really hard to catch errors, try this neat trick: We are going to define a helper class which allows the derived classes to "freeze" their attributes. Whenever an attribute is added after the freezing, a TypeError is thrown. 

In [ ]:
class FrozenClass():
    __isfrozen = False

    def __setattr__(self, key, value):
        # check if attribute exists and if class is frozen
        if self.__isfrozen and not hasattr(self, key):
            raise TypeError("%r is a frozen class" % self)
        else:
            object.__setattr__(self, key, value)

    def _freeze(self):
        self.__isfrozen = True

In [ ]:
class FrozenFoo(FrozenClass):
    def __init__(self):
        self.fluorescence = None
        self._freeze()
        
    def get_surprise(self):
        if self.fluorescence is None:
            number = np.random.rand(1)
            self.flourescence = number
        else:
            pass

In [ ]:
c = FrozenFoo()
print(c.fluorescence)
c.get_surprise()
print(c.fluorescence)

If I had just known this earlier... I found it late, but [here](https://stackoverflow.com/questions/3603502/prevent-creating-new-attributes-outside-init).

In case you wonder, this can also be done using an decorator with @. And in case you wonder what a decorator actually is:

## 7. Decorators

Decorators allow you to extend and modify the behavior of a callable (functions, methods and classes) _without_ permanently modifying the callable itself. Usage example:

* logging
* access control and authentification
* instrumentation and timing
* ...

We need to know two things in order to understand how decorators can work:

<div class="alert alert-success">
    <strong>1. Lesson learned</strong>: Functions are first-class: they can be assigned to variables and passed to and from other functions. <p>
    <strong>2. Lesson learned</strong>: Functions can be defined inside other functions: the child function has access to the parent function's local state ("lexical closure").
</div>

OK, let's start writing some code:

In [ ]:
def my_decorator(func):
    print(f'Calling {func} now..')
    return func

In [ ]:
def greet():
    return 'Hello'

In [ ]:
greet()

In [ ]:
my_decorator(greet)()

Cool, but also somewhat bulky. How about this nice _syntactic sugare_ @?

In [ ]:
def my_new_decorator(func):
    def wrapper():
        print(f'Calling {func} now..')
        return func()
    return wrapper

In [ ]:
@my_new_decorator
def greet():
    return 'Hello'

In [ ]:
greet()

OK, nice, but what if I need function arguments? \*args and \*\*kwargs to the rescue!

In [ ]:
def my_better_decorator(func):
    def wrapper(*args, **kwargs):
        print(f'Calling {func} with {args} and {kwargs} now..')
        return func(*args, *kwargs)
    return wrapper

In [ ]:
@my_better_decorator
def greet_person(person):
    return f'Hi {person}!'

greet_person('Bob')

One caveat, though: Using decorators like this looses metadata like docstrings! E.g.:

In [ ]:
def greet_person(person):
    """A friendly greeting"""
    return f'Hi {person}!'

In [ ]:
print(greet_person.__doc__)

In [ ]:
print(my_better_decorator(greet_person).__doc__)

Thankfully there is a quick fix for that: the `functools.wraps` decorator copies over lost metadata from the undecorated function to the decorator closure:

In [ ]:
from functools import wraps

def my_even_better_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f'Calling {func} with {args} and {kwargs} now..')
        return func(*args, *kwargs)
    return wrapper

In [ ]:
print(my_even_better_decorator(greet_person).__doc__)

So, let's now revise the `FrozenClass` from above:

In [ ]:
def froze_it(cls):
    cls.__frozen = False

    def frozensetattr(self, key, value):
        if self.__frozen and not hasattr(self, key):
            raise TypeError(f"Class {cls.__name__} is frozen. Cannot set {key} = {value}")
        else:
            object.__setattr__(self, key, value)

    def init_decorator(func):
        @wraps(func)
        def wrapper(self, *args, **kwargs):
            func(self, *args, **kwargs)
            self.__frozen = True
        return wrapper

    cls.__setattr__ = frozensetattr
    cls.__init__ = init_decorator(cls.__init__)

    return cls

Looks a bit more complex, right? But remember, you just have to write this (or copy it from the internet) once, in contrast to your actual classes you want to freeze:

In [ ]:
@froze_it
class Foo():
    def __init__(self):
        self.fluorescence = None
        
    def get_surprise(self):
        if self.fluorescence is None:
            number = np.random.rand(1)
            self.flourescence = number
        else:
            pass

In [ ]:
c = Foo()
print(c.fluorescence)
c.get_surprise()
print(c.fluorescence)

Much simpler! Note however, that freezing will always happen after the `__init__` routine. This is also true for derived classes, so you need to add attributes before calling `super` there.

In [ ]:
class Bar(Foo):
    def __init__(self):
        super().__init__()
        self.data = self.fluorescence

In [ ]:
b = Bar()

<div class="alert alert-success">
    <strong>Lesson learned</strong>: decorators define reusable building blocks to modify behaviors of callables without modifying the callable itself. As usual, they are not a silver bullet for all problems, though.
</div>

OK, one last bit on decorators: You have probably seen them in the context of numba. Let's see what they can do for you. 

## 8. Make Python fast (again)

Many of the things here are taken from [this website](https://www.ibm.com/developerworks/community/blogs/jfp/entry/Python_Meets_Julia_Micro_Performance?lang=en). Consider this well-known example:

In [ ]:
def fib(n):
    if n<2:
        return n
    return fib(n-1)+fib(n-2)

In [ ]:
t_original = %timeit -o fib(20)

In [ ]:
t_original.best


Hm, could be slow, could be fast, who knows.. but let's try [numba](http://numba.pydata.org/):

In [ ]:
from numba import jit

In [ ]:
@jit
def fib_jit(n):
    if n < 2:
        return n
    return fib_jit(n - 1)+fib_jit(n - 2)

In [ ]:
t_jit = %timeit -o fib_jit(20)

In [ ]:
print(f'Speedup over naive code is: {t_original.best / t_jit.best:8.2f}')

Is recursion causing us trouble here?

In [ ]:
@jit
def fib_norec(n):
    if n < 2:
        return n
    a, b = 1, 0
    for i in range(n - 1):
        a, b = a + b, a
    return a  

In [ ]:
t_norec = %timeit -o fib_norec(20)

In [ ]:
print(f'Speedup over naive code is: {t_original.best / t_norec.best:8.2f}')

Pretty impressive, eh? And we did not even use numba's advanced features like GPUs, vectorization etc.. But there is one additional way to speed this up even more, using decorators!

In [ ]:
from functools import lru_cache as cache

@cache(maxsize=None)
def fib_cache(n):
    if n < 2:
        return n
    a, b = 1, 0
    for i in range(n - 1):
        a, b = a + b, a
    return a 

In [ ]:
t_cache = %timeit -o fib_cache(20)

In [ ]:
print(f'Speedup over naive code is: {t_original.best / t_cache.best:8.2f}')

Holy cow! I just added caching to the function. OK, point taken, this is because of the recursive nature of the function, but still.

And in case you wonder what [Cython](https://cython.org/) will do with it (you know, the thing that can make C code out of your Python code, if you do it right):

In [ ]:
%load_ext Cython

In [ ]:
%%cython -a
def fib_cython(n):
    if n < 2:
        return n
    a, b = 1, 0
    for i in range(n - 1):
        a, b = a + b, a
    return a 

In [ ]:
t_cython = %timeit -o fib_cython(20)

In [ ]:
print(f'Speedup over naive code is: {t_original.best / t_cython.best:8.2f}')

Hm, ok, not bad. How about static typing?

In [ ]:
%%cython -a
cpdef long fib_cython_type(long n):
    if n < 2:
        return n
    cdef int a = 1
    cdef int b = 0
    for i in range(n - 1):
        a, b = a + b, a
    return a  

In [ ]:
t_cython_type = %timeit -o fib_cython_type(20)

In [ ]:
print(f'Speedup over naive code is: {t_original.best / t_cython_type.best:8.2f}')

<div class="alert alert-success">
    <strong>Lesson learned</strong>: If you need fast code, spend some time not writing naive code. Use built-in functionality, use Numpy/Scipy, and/or use Numba and Cython if you can. Also, exploit the structure of your problem!
</div>

## 9. Three more nasty mistakes

And now: some subtle, harder-to-catch mistakes that can bite even some more advanced Python developers in the rear! See [this website](https://www.toptal.com/python/top-10-mistakes-that-python-programmers-make) for reference.

### 1. Misusing expressions as defaults for function arguments

In [41]:
def foo(bar=[]):
    bar.append('baz')
    return bar

In [42]:
foo()

['baz']

In [43]:
foo()

['baz', 'baz']

<div class="alert alert-success">
    <strong>Lesson learned</strong>: The default value for a function argument is only evaluated once, at the time that the function is defined. 
</div>

### 2. Using class variables incorrectly

In [44]:
class A():
    x = 1

class B(A):
    pass

class C(A):
    pass

In [45]:
print(A.x, B.x, C.x)

1 1 1


In [46]:
B.x = 2
print(A.x, B.x, C.x)

1 2 1


In [48]:
A.x = 3
print(A.x, B.x, C.x)

3 2 3


<div class="alert alert-success">
    <strong>Lesson learned</strong>: Class variables are internally handled as dictionaries and follow what is often referred to as "Method Resolution Order (MRO)". C.x is just a reference to A.x here.
</div>

### 3. Misunderstanding Python scope rules

In [ ]:
lst = [1, 2, 3]

def foo():
    lst.append(4)
    
def bar():
    lst += [5]

In [ ]:
foo()
print(lst)

In [ ]:
bar()
print(lst)

<div class="alert alert-success">
    <strong>Lesson learned</strong>: When you make an assignment to a variable in a scope, that variable is automatically considered by Python to be local to that scope and shadows any similarly named variable in any outer scope. 
</div>

Want to find out what actually happens in your code? Disassemble!

In [ ]:
import dis
dis.dis(foo)

In [ ]:
dis.dis(bar)

<div class="alert alert-success">
    <strong>Lesson learned</strong>: Use the built-in module dis to inspect the bytcode of Python function. This can be quite revealing.. or disturbing.
</div>

## Final words

That's it! There is much more to learn and see out there. The best way to do that is by.. well.. **doing**. Play with Python, test out things you are interested in. Don't just read about it, don't just listen to people (like me). **Do it yourself**. 

You can find this notebook at [Github](https://github.com/pancetta/fun_with_python/blob/master/python_tricks_lecture.ipynb)! Have **fun**!